In [1]:
import sys
from absl import flags
sys.argv=['preserve_unused_tokens=False']
flags.FLAGS(sys.argv)

['preserve_unused_tokens=False']

In [2]:
pip install bert-tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Downloading tokenization script created by the Google
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split

from bert import tokenization

In [6]:
img_feature = np.load('Image_Embeddings/Emb_feature/new_test_features/.npy')
np.shape(img_feature)

In [ ]:
FMS = pd.read_csv("FMS_final.csv")
FMS['textNdesc'] = 'In the picture '+ FMS.gen_caption + ' And the text says: ' + FMS.text
print(FMS.textNdesc[0], '\n\n')
print(len(FMS))

In the picture man in black and white cap is holding up sign . And the text says: its their character not their color that matters 


8313


In [5]:
dev_img_feature = np.load('Image_Embeddings/Emb_feature/dev_test_features/final_images.npy')
np.shape(dev_img_feature)

(494, 100, 1024)

In [6]:
dev_FMS = pd.read_csv("FMS_final_dev.csv")
dev_FMS['textNdesc'] = 'In the picture '+ dev_FMS.gen_caption + ' And the text says: ' + dev_FMS.text
print(dev_FMS.textNdesc[0], '\n\n')
print(len(dev_FMS))

In the picture man in blue shirt and white shirt is standing in front of store . And the text says: voila! fresh batch of baked goods ! 


494


In [7]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [8]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    image_input = tf.keras.Input(
    shape=(100,1024),
    batch_size=None,
    name="image_input",
)

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])


    

    concatted = tf.keras.layers.Concatenate(axis = 1)([sequence_output, image_input])


    clf_output = concatted[:, 0, :]

    hidden_layer1 = Dense(1024, activation='relu')(clf_output)
    Dropout_layer1 = Dropout(.2)(hidden_layer1)

    hidden_layer2 = Dense(2048, activation='relu')(Dropout_layer1)
    Dropout_layer2 = Dropout(.3)(hidden_layer2)

    hidden_layer3 = Dense(1024, activation='relu')(Dropout_layer2)
    Dropout_layer3 = Dropout(.2)(hidden_layer3)
    

    # concatted = tf.keras.layers.Concatenate()([clf_output, image_input])

    out = Dense(1, activation='sigmoid')(Dropout_layer3)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids, image_input], outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy', 'AUC'])
    
    return model

In [9]:
%%time
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=False)

Wall time: 14.6 s


In [10]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [12]:
train_input = bert_encode(dev_FMS.textNdesc.values, tokenizer, max_len=100)
train_labels = dev_FMS.label.values

In [13]:
test_input = bert_encode(dev_FMS.textNdesc.values, tokenizer, max_len=160)
test_labels = dev_FMS.label.values

In [17]:
model = build_model(bert_layer, max_len=160)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 160)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 160)]        0           []                               
                                                                                                  
 segment_ids (InputLayer)       [(None, 160)]        0           []                               
                                                                                                  
 keras_layer (KerasLayer)       multiple             335141889   ['input_word_ids[0][0]',         
                                                                  'input_mask[0][0]',       

c:\Users\yasir\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [13]:
%%time
checkpoint = ModelCheckpoint('model3.h5', monitor='val_loss', save_best_only=True)

train_history = model.fit(
    [train_input, img_feature], train_labels,
    validation_split=0.3,
    epochs=200,
    callbacks=[checkpoint],
    batch_size=128,
    steps_per_epoch=12   
)

Epoch 1/200
12/12 [==============================] - 1216s 104s/step - loss: 0.6799 - accuracy: 0.5892 - auc: 0.4798 - val_loss: 0.6963 - val_accuracy: 0.6055 - val_auc: 0.4512
Epoch 2/200
12/12 [==============================] - 1124s 99s/step - loss: 0.6592 - accuracy: 0.6510 - auc: 0.5024 - val_loss: 0.6853 - val_accuracy: 0.6055 - val_auc: 0.5081
Epoch 3/200
12/12 [==============================] - 1116s 98s/step - loss: 0.6549 - accuracy: 0.6400 - auc: 0.5273 - val_loss: 0.6676 - val_accuracy: 0.6055 - val_auc: 0.5974
Epoch 4/200
12/12 [==============================] - 1081s 95s/step - loss: 0.6425 - accuracy: 0.6687 - auc: 0.5068 - val_loss: 0.6720 - val_accuracy: 0.6055 - val_auc: 0.6302
Epoch 5/200
12/12 [==============================] - 1175s 104s/step - loss: 0.6456 - accuracy: 0.6517 - auc: 0.5387 - val_loss: 0.6673 - val_accuracy: 0.6055 - val_auc: 0.6491
Epoch 6/200
12/12 [==============================] - 1473s 129s/step - loss: 0.6419 - accuracy: 0.6576 - auc: 0.5435 -

In [ ]:
test_data = pd.read_json("Data/dev.jsonl",lines=True)
test_data

In [18]:
model.load_weights('model3.h5')
test_pred = model.predict([test_input, dev_img_feature])

16/16 [==============================] - 247s 15s/step


In [19]:
from sklearn.metrics import roc_auc_score, balanced_accuracy_score
y_actual = list(test_labels)
y_prob = list(test_pred.reshape(len(test_pred), ))
print("AUC: ", roc_auc_score(y_actual, y_prob))

AUC:  0.632605559926567
